In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import time
import numpy as np

start_time = time.time()

# Initialize MediaPipe Holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils  # For drawing landmarks

# Open webcam (0 = default camera)
cap = cv2.VideoCapture(0)

# Get Video FPS and Frame Count
fps = cap.get(cv2.CAP_PROP_FPS)
frame_skip = 2  # Process every 2nd frame (change as needed)

# Initialize storage
landmark_data = []
frame_num = 0

# Load MediaPipe Holistic
with mp_holistic.Holistic(static_image_mode=False, 
                          min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit when video ends

        frame_num += 1

        # Skip frames to improve processing speed
        if frame_num % frame_skip != 0:
            continue

        # Convert to RGB (Keep 1 channel if needed)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process frame with MediaPipe
        results = holistic.process(rgb_frame)

        # Store landmarks
        def save_landmarks(landmarks, type_name):
            if landmarks:
                landmark_data.extend([
                    [frame_num, f"{frame_num}_{type_name}_{i}", type_name, i, lm.x, lm.y, lm.z]
                    for i, lm in enumerate(landmarks.landmark)
                ])

        # Extract landmarks (without face)
        save_landmarks(results.pose_landmarks, "pose")
        save_landmarks(results.left_hand_landmarks, "left_hand")
        save_landmarks(results.right_hand_landmarks, "right_hand")

        # Draw landmarks on the frame
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Show the camera feed with landmarks
        cv2.imshow("Holistic Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Save the data to a CSV file using NumPy for efficiency
df = pd.DataFrame(landmark_data, columns=["frame", "row_id", "type", "landmark_index", "x", "y", "z"])
df.to_csv("landmarks_output_Withoutface.csv", index=False)

end_time = time.time()
execution_time = end_time - start_time
print(f"✅ Landmark extraction completed! Data saved to 'landmarks_output_Withoutface.csv'.")
print(f"🚀 Execution Time: {execution_time:.6f} seconds")


I0000 00:00:1743286809.136253  700287 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743286809.226341  701673 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743286809.241262  701675 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743286809.243623  701678 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743286809.243651  701679 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743286809.243688  701675 inference_feedback_manager.cc:114] Feedback manager requires a mod

✅ Landmark extraction completed! Data saved to 'landmarks_output_Withoutface.csv'.
🚀 Execution Time: 5.129965 seconds
